# Proyecto Adicional
## Dante Saavedra

## Importaciones y configuración de la cuenta twitter y neo4j

In [20]:
from twython import Twython
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
import pandas as pd
import pprint as pp
APP_KEY = 'rXbK97bg6LvBpH9WqDJkqw10T'
APP_SECRET = 'nRGOOhI3sCzsNMLpd5drXmHRAhvT9BOi9HN2vtCI3975CUZUHe'

twitter = Twython(APP_KEY, APP_SECRET)

OAUTH_TOKEN = '976279920279261184-f5RUiD4PRLmOzVQV8FzfXJ827bKwLvI'
OAUTH_TOKEN_SECRET = 'sUk7XhsexQWJ6cmcu3cSs8YgBWYSvFBbjiMrazXTgpNs0'

twitter = Twython(APP_KEY, APP_SECRET,OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
db = GraphDatabase("http://localhost:7474", username="neo4j", password="root")

## Cargar Tweet

In [16]:
resultados =twitter.search(q='ValdiviaCL', count =100)
datos = []
for i in range(0,len(resultados["statuses"])):
    menciones =[]
    for j in range(0,len(resultados["statuses"][i]['entities']['user_mentions'])):
        menciones.append({'usuario': resultados['statuses'][i]['entities']['user_mentions'][j]['name']})
    dato = {
        'texto': resultados['statuses'][i]['text'],
        'usuario': resultados['statuses'][i]['user']['name'],
        'menciones': menciones
    }
    datos.append(dato)

### Listados de Usuarios

In [25]:
usuarios = []
usuariosGraph = []
usuariosNeo = db.labels.create("Usuarios")
for d in datos:
    for c in d['menciones']:
        usuarios.append(c['usuario'])
    usuarios.append(d['usuario'])
usuarios = list(set(usuarios))

### Listado de Tweets

In [23]:
tweets = []
tweetsGraph = []
tweetsNeo = db.labels.create("Tweets")
for d in datos:
    tweets.append(d['texto'])

## Cargar Personas y Tweets

In [26]:
for u in range(len(usuarios)):
    usuariosGraph.append(db.nodes.create(Usuario=usuarios[u]))
    usuariosNeo.add(usuariosGraph[u])
for t in range(len(tweets)):
    tweetsGraph.append(db.nodes.create(Tweet=tweets[t]))
    tweetsNeo.add(tweetsGraph[t])

## Relacionar Tweet con Persona

In [29]:
for d in datos:
    iTweet = tweets.index(d['texto'])
    iUsuario = usuarios.index(d['usuario'])
    tweetsGraph[iTweet].relationships.create("Posteado por", usuariosGraph[iUsuario])
    for m in d['menciones']:
        iMencionado = usuarios.index(m['usuario'])
        tweetsGraph[iTweet].relationships.create("Menciona a", usuariosGraph[iMencionado])